In [ ]:
import os,re,random,time,sys
import numpy as np
import cv2
import SimpleITK as sitk
from matplotlib import pyplot as plt
import pandas as pd
from skimage.measure import regionprops,label
from time import time
import os,re,random
from tqdm import tqdm
from math import sqrt
from PIL import Image
from skimage import exposure
import random 

## load all the data

In [ ]:
#contains index (unique token of the image), path_recombined_img, path_low_energy_img, path_mask_img, outcome
df_train = pd.read_excel('all_info_train_dataset.xlsx',index=True)

## making the images

In [ ]:
def fct_generate_figures(temp_img):
    plt.figure(figsize=(20, 20))
    plt.imshow(temp_img, cmap='gray')
    plt.show()
    fig1 = sns.displot(temp_img.flatten(), element="step",bins=50)
    fig1.fig.set_figwidth(11)
    fig1.fig.set_figheight(8)

In [ ]:
def crop_img_experiment_with_sergey(img_read_re, img_read_le, msk_read,generate_figures=None):
    otsu = sitk.OtsuThresholdImageFilter()
    otsu_image = otsu.Execute(img_read_re)
    otsu_array = sitk.GetArrayFromImage(otsu_image)
    
    invert_otsu = (np.ones(otsu_array.shape)-otsu_array).astype(np.uint8)
    (contours,_) = cv2.findContours(invert_otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    max_ctr = contours[0]
    for ctr in contours:
        if cv2.contourArea(ctr) > cv2.contourArea(max_ctr):
            max_ctr= ctr
    img_temp = np.zeros(invert_otsu.shape)
    polygon = ctr
    cv2.fillPoly( img_temp, [max_ctr], [1] )
    otsu_array = np.ones(invert_otsu.shape)-img_temp
    
    if generate_figures=='le' or generate_figures=='re':
        fct_generate_figures(img_temp)
        
    temp_img_re = sitk.GetArrayFromImage(img_read_re)
    
    if generate_figures=='re':
        fct_generate_figures(temp_img_re)
    
    temp_img_le_original = sitk.GetArrayFromImage(img_read_le)
    
    if generate_figures=='le':
        fct_generate_figures(temp_img_le_original)
        
    temp_msk = sitk.GetArrayFromImage(msk_read)
    temp_img_re =(np.ones((img_read_re.GetSize()[1], img_read_re.GetSize()[0]))-otsu_array)*temp_img_re
    temp_img_le =(np.ones((img_read_le.GetSize()[1], img_read_le.GetSize()[0]))-otsu_array)*temp_img_le_original
    props = regionprops(np.array(temp_img_re>0,np.uint8))
    r0, c0, r1, c1 = props[0].bbox
    temp_img_re = temp_img_re[r0:r1, c0:c1]
    
    if generate_figures=='re':
        fct_generate_figures(temp_img_re)
    
    if generate_figures=='re':
        fct_generate_figures(temp_img_re)
    
    temp_img_re = pre_processing_for_img(temp_img_re)
    
    if generate_figures=='re':
        fct_generate_figures(temp_img_re)
    
    temp_img_le = temp_img_le[r0:r1, c0:c1]
    
    if generate_figures=='le':
        fct_generate_figures(temp_img_le)
    
    temp_img_le = pre_processing_for_img(temp_img_le)
    
    if generate_figures=='le':
        fct_generate_figures(temp_img_le)

    
    temp_msk =temp_msk[r0:r1, c0:c1]

    return temp_img_re, temp_img_le,temp_msk

In [ ]:
def resample_intensities(orig_img,bin_nr=256):            ## Intensity resampling whole image/or masked region
    v_count=0
    img_list=[]
    filtered = orig_img.copy()
    if np.min(orig_img.flatten())<0:
        filtered+=np.min(orig_img.flatten())
    resampled = np.zeros_like(filtered)
    max_val_img = np.max(filtered.flatten())
    min_val_img = np.min(filtered.flatten())
    step = (max_val_img-min_val_img)/bin_nr

    for st in np.arange(step+min_val_img,max_val_img+step,step):
        resampled[(filtered<=st)&(filtered>=st-step)] = v_count
        v_count+=1
    
    print("Resampling with a step of: ",step ,'Amount of unique values, original img: ',len(np.unique(orig_img.flatten())),'resampled img: ',len(np.unique(resampled.flatten())))
    return np.array(resampled,dtype=np.uint16)

In [ ]:
def pre_processing_for_img(img):
    temp_img = img.copy()
    low_thr = np.quantile(temp_img[temp_img>0], 0.01)
    high_thr = np.quantile(temp_img[temp_img>0], 0.99)
    temp_img[temp_img<low_thr] = low_thr
    temp_img[temp_img>high_thr] = high_thr
    if len(np.unique(temp_img[temp_img>0])) > 256:
            temp_img_sampled = resample_intensities(temp_img[temp_img>0])
            temp_img[temp_img>0] = temp_img_sampled
    else:
        new_img =(temp_img-np.min(temp_img))/(np.max(temp_img)-np.min(temp_img)) 
        temp_img = (new_img*255).astype(np.uint8)
    return temp_img

In [ ]:
import seaborn as sns
def load_patient_experiment(dataframe, filename,generate_figures=None):
    img_read_re = sitk.ReadImage(list(dataframe[dataframe['Unnamed: 0'] == filename]['path_recombined_img'])[0])
    img_read_le = sitk.ReadImage(list(dataframe[dataframe['Unnamed: 0'] == filename]['path_low_energy_img'])[0])
    #img_read_le = img_read_le[:,:,0] 
    msk_read = sitk.ReadImage(list(dataframe[dataframe['Unnamed: 0'] == filename]['path_mask_img'])[0])

    img_re, img_le, msk = crop_img_experiment_with_sergey(img_read_re, img_read_le, msk_read,generate_figures)
    props_msk = regionprops(np.array(msk>0,np.uint8))
    final_msk = np.array(msk>0,np.uint8)
    is_empty = 1 - np.any(final_msk).astype(int)
    return img_le,img_re, props_msk, final_msk,is_empty

In [ ]:
def measure_clipped_values(img_read_re, img_read_le):
    otsu = sitk.OtsuThresholdImageFilter()
    otsu_image = otsu.Execute(img_read_re)
    otsu_array = sitk.GetArrayFromImage(otsu_image)
    
    invert_otsu = (np.ones(otsu_array.shape)-otsu_array).astype(np.uint8)
    (contours,_) = cv2.findContours(invert_otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    max_ctr = contours[0]
    for ctr in contours:
        if cv2.contourArea(ctr) > cv2.contourArea(max_ctr):
            max_ctr= ctr
    img_temp = np.zeros(invert_otsu.shape)
    polygon = ctr
    cv2.fillPoly( img_temp, [max_ctr], [1] )
    otsu_array = np.ones(invert_otsu.shape)-img_temp
    
    temp_img_re = sitk.GetArrayFromImage(img_read_re)
    temp_img_le = sitk.GetArrayFromImage(img_read_le)
    array_air_re = otsu_array*temp_img_re
    median_re = np.median(array_air_re[array_air_re > 0])
    array_air_le = otsu_array*temp_img_le
    median_le = np.median(array_air_le[array_air_le > 0])
    temp_img_re =(np.ones((img_read_re.GetSize()[1], img_read_re.GetSize()[0]))-otsu_array)*temp_img_re
    temp_img_le =(np.ones((img_read_le.GetSize()[1], img_read_le.GetSize()[0]))-otsu_array)*temp_img_le
    temp_img_re[temp_img_re == 0] = median_re
    low_thr_re = np.quantile(temp_img_re[temp_img_re>median_re], 0.01)
    high_thr_re = np.quantile(temp_img_re[temp_img_re>median_re], 0.99)
    temp_img_re[temp_img_re<low_thr_re] = low_thr_re
    temp_img_re[temp_img_re>high_thr_re] = high_thr_re
    
    temp_img_le[temp_img_le == 0] = median_le
    low_thr_le = np.quantile(temp_img_le[temp_img_le>median_le], 0.01)
    high_thr_le = np.quantile(temp_img_le[temp_img_le>median_le], 0.99)
    temp_img_le[temp_img_le<low_thr_le] = low_thr_le #maybe should be better adapted, with the mean of pixels below 90% of the breast values 
    temp_img_le[temp_img_le>high_thr_le] = high_thr_le
    
    return [low_thr_re,high_thr_re],[low_thr_le,high_thr_le]

### training set

In [ ]:
clahe = cv2.createCLAHE(clipLimit =2.5, tileGridSize=(16,16))
clahe_recombined_2 = cv2.createCLAHE(clipLimit =1.0, tileGridSize=(16,16)) #are they the best parameters?
ext_filenames = list(df_train['Unnamed: 0'])
list_coord = []
list_train_not_done = []
for i,filename in tqdm(enumerate(ext_filenames)):
    generate_figures=False
    img_le, img_re, props_msk,final_msk,is_empty = load_patient_experiment(df_train,filename,generate_figures)
    if len(props_msk)==0 and is_empty==1: #we select only the images which don't contains a mask
        new_img_le =(img_le-np.min(img_le))/(np.max(img_le)-np.min(img_le)) 
        im_le = (new_img_le*255).astype(np.uint8)

        x_le = clahe.apply(im_le).astype(np.uint8)
        new_img_re =(img_re-np.min(img_re))/(np.max(img_re)-np.min(img_re)) 
        new_img_re = (new_img_re*255).astype(np.uint8)
        
#         if generate_figures=='le':
#             fct_generate_figures(im_le)
            
#         if generate_figures=='re':
#             fct_generate_figures(new_img_re)
        
        x_re = clahe.apply(new_img_re).astype(np.uint8)
        
#         if generate_figures=='re':
#             fct_generate_figures(x_re)

        x_re_2 = clahe_recombined_2.apply(new_img_re).astype(np.uint8)
        
#         if generate_figures=='re':
#             fct_generate_figures(x_re_2)
        
        temp_im_le = Image.fromarray(x_le)
        temp_im_re = Image.fromarray(x_re)
        temp_im_re_2 = Image.fromarray(x_re_2)
        #temp_mask_png = Image.fromarray(mask2)
        temp_name_le = ##
        temp_name_re = ##
        #temp_name_mask = ##
        temp_name_re_2 =##

#         temp_im_le.save(temp_name_le)
#         temp_im_re.save(temp_name_re)
#         temp_im_re_2.save(temp_name_re_2)
        #temp_mask_png.save(temp_name_mask)
        list_coord.append([r""+",,,,,"+str(list(df_train[df_train['Unnamed: 0'] == filename]['outcome'])[0])+","])
    else:
        list_train_not_done.append(filename)
        print(filename)

In [ ]:
df = pd.DataFrame(redone_list, columns=["filename","x1","y1","x2","y2","labels","path_mask"])
df.to_csv("annotations_train_dataset.csv",index=False)

In [ ]:
ext_filenames = list(df_train['Unnamed: 0'])

for i,filename in tqdm(enumerate(ext_filenames)):
    temp_name_le = "low_energy_to_jpg\\"+filename+".jpg"
    temp_name_re = "recombined_to_jpg\\"+filename+".jpg"
    temp_name_re_2 = "recombined_2_to_jpg\\"+filename+".jpg"
    temp_name_3channels = "colored_to_jpg\\" + filename + ".jpg"

    if os.path.exists(temp_name_re):
        r = cv2.imread(temp_name_le,0)
        g= cv2.imread(temp_name_re,0)
        b =cv2.imread(temp_name_re_2,0)
        img = cv2.merge((r,g,b))

        cv2.imwrite(temp_name_3channels,img)